In [1]:
from collections import defaultdict

In [2]:
f = open('day16_input.txt')
lines = f.readlines()
f.close()
data = [(line[0:-1]) for line in lines]

In [3]:
print(f'The data looks like this:')
data[0:30]

The data looks like this:


['departure location: 36-626 or 651-973',
 'departure station: 38-134 or 142-966',
 'departure platform: 32-465 or 489-972',
 'departure track: 40-420 or 446-973',
 'departure date: 38-724 or 738-961',
 'departure time: 30-358 or 377-971',
 'arrival location: 48-154 or 166-965',
 'arrival station: 48-669 or 675-968',
 'arrival platform: 27-255 or 276-965',
 'arrival track: 37-700 or 720-955',
 'class: 50-319 or 332-958',
 'duration: 35-822 or 835-949',
 'price: 40-791 or 802-951',
 'route: 42-56 or 82-968',
 'row: 40-531 or 555-968',
 'seat: 49-681 or 695-962',
 'train: 31-567 or 593-953',
 'type: 42-840 or 855-949',
 'wagon: 31-165 or 176-962',
 'zone: 48-870 or 896-970',
 '',
 'your ticket:',
 '127,89,149,113,181,131,53,199,103,107,97,179,109,193,151,83,197,101,211,191',
 '',
 'nearby tickets:',
 '835,933,819,240,276,334,830,786,120,791,301,770,249,767,177,84,838,85,596,352',
 '193,697,654,130,5,907,754,925,817,663,938,595,930,868,56,128,598,197,381,452',
 '922,462,747,775,599,787,76

In [4]:
rules = data[0:20]
my_ticket = data[22]
nearby_tickets = data[25:]

In [5]:
print(f'There are {len(nearby_tickets)} nearby tickets.')

There are 242 nearby tickets.


# Part 1

Part 1 is to check which tickets are invalid. By "invalid", it means at least one of those numbers on that ticket doesn't fit any of the fields listed as the rules. For example, if an entry has the number 2, it won't be fit any of the fields, and we'll call that entry an invalid ticket.

As the first step, let me reformat the "rules":

In [6]:
zones = []
for rule in rules:
    elements = rule.split(' ')
    zone1 = elements[-3].split('-')
    zones.append([int(zone1[0]), int(zone1[1])])
    zone2 = elements[-1].split('-')
    zones.append([int(zone2[0]), int(zone2[1])])

print(zones)

[[36, 626], [651, 973], [38, 134], [142, 966], [32, 465], [489, 972], [40, 420], [446, 973], [38, 724], [738, 961], [30, 358], [377, 971], [48, 154], [166, 965], [48, 669], [675, 968], [27, 255], [276, 965], [37, 700], [720, 955], [50, 319], [332, 958], [35, 822], [835, 949], [40, 791], [802, 951], [42, 56], [82, 968], [40, 531], [555, 968], [49, 681], [695, 962], [31, 567], [593, 953], [42, 840], [855, 949], [31, 165], [176, 962], [48, 870], [896, 970]]


to combine all the zones (apparently there are many overlaps), use two-pointers:

In [7]:
zones.sort()
left, right = zones[0]
combined_zones = []
for zone in zones[1:]:
    new_left, new_right = zone
    if new_left > right:
        combined_zones.append([left, right])
        left, right = new_left, new_right
    else:
        right = max(right, new_right)
combined_zones.append([left, right])
print(combined_zones)

[[27, 973]]


So instead of several small zones, there is actually only one zone.

In [8]:
invalid_nums_sum = 0
good_tickets = []
for i, ticket in enumerate(nearby_tickets):
    good_ticket = True
    for num in ticket.split(','):
        if not 27 < int(num) < 973:  # or we can loop over all the zones, in case there are more than one in the combined_zones
            invalid_nums_sum += int(num)
            good_ticket = False
    if good_ticket:
        good_tickets.append(i)

print(invalid_nums_sum)

30869


In [9]:
print(f'There are {len(good_tickets)} good tickets.')

There are 190 good tickets.


# Part 2

By doing what we did in part 1, we found out all the valid tickets. Now for part 2, we need to look at those good tickets, and try to figure out which field is which.

We can't look at all the rules as a whole now, instead:

In [10]:
rules_dict = {}
for rule in rules:
    elements = rule.split(':')
    key = elements[0]
    zone1, zone2 = elements[1].split(' or ')
    zone1 = [int(n) for n in zone1.split('-')]
    zone2 = [int(n) for n in zone2.split('-')]
    rules_dict[key] = [zone1, zone2]
rules_dict

{'departure location': [[36, 626], [651, 973]],
 'departure station': [[38, 134], [142, 966]],
 'departure platform': [[32, 465], [489, 972]],
 'departure track': [[40, 420], [446, 973]],
 'departure date': [[38, 724], [738, 961]],
 'departure time': [[30, 358], [377, 971]],
 'arrival location': [[48, 154], [166, 965]],
 'arrival station': [[48, 669], [675, 968]],
 'arrival platform': [[27, 255], [276, 965]],
 'arrival track': [[37, 700], [720, 955]],
 'class': [[50, 319], [332, 958]],
 'duration': [[35, 822], [835, 949]],
 'price': [[40, 791], [802, 951]],
 'route': [[42, 56], [82, 968]],
 'row': [[40, 531], [555, 968]],
 'seat': [[49, 681], [695, 962]],
 'train': [[31, 567], [593, 953]],
 'type': [[42, 840], [855, 949]],
 'wagon': [[31, 165], [176, 962]],
 'zone': [[48, 870], [896, 970]]}

In [11]:
def num_fit_field(num: int, field: str, rules_dict: dict) -> bool:
    zone1 = rules_dict[field][0]
    zone2 = rules_dict[field][1]
    return (zone1[0]<=num<=zone1[1]) or (zone2[0]<=num<=zone2[1])    

In [12]:
possible_fields = defaultdict(lambda: set())
for i in good_tickets:
    ticket = nearby_tickets[i]
    current_possible_fields = defaultdict(lambda: set())
    for j, num in enumerate(ticket.split(',')):
        for field in rules_dict:
            if num_fit_field(int(num), field, rules_dict):
                current_possible_fields[field].add(j)
    if i == good_tickets[0]:
        possible_fields = current_possible_fields
    else:
        for field in possible_fields:
            possible_fields[field] = possible_fields[field].intersection(current_possible_fields[field])

In [13]:
for k, v in possible_fields.items():
    print(f'The "{k}" field can be at the idx:')
    print(v)

The "departure location" field can be at the idx:
{1, 3, 4, 8, 9, 10, 11, 12, 13, 17, 18}
The "departure station" field can be at the idx:
{1, 3, 4, 8, 9, 10, 12, 13, 18}
The "departure platform" field can be at the idx:
{1, 3, 4, 9, 10, 12, 13, 18}
The "departure track" field can be at the idx:
{1, 3, 4, 8, 9, 10, 11, 12, 13, 18}
The "departure date" field can be at the idx:
{0, 1, 3, 4, 8, 9, 10, 11, 12, 13, 17, 18, 19}
The "departure time" field can be at the idx:
{1, 3, 4, 8, 9, 10, 11, 12, 13, 17, 18, 19}
The "arrival location" field can be at the idx:
{18, 3, 12, 13}
The "arrival station" field can be at the idx:
{0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19}
The "arrival platform" field can be at the idx:
{1, 3, 4, 9, 12, 13, 18}
The "arrival track" field can be at the idx:
{1, 3, 4, 12, 13, 18}
The "class" field can be at the idx:
{0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19}
The "duration" field can be at the idx:
{3, 4, 12, 13, 18}
The "price" 

We can comb through what we got above and get one-to-one correspondence:

For example, from the following two:
- The "seat" field can be at the idx:
{18, 12}
- The "train" field can be at the idx:
{18}

we know that the "seat" field has to be at idx 12.

In [14]:
changed = set()
while len(changed) < 20:   # there are 20 fields in total
    for key in possible_fields:
        if len(possible_fields[key]) == 1:          
            changed.add(list(possible_fields[key])[0])
        else:
            for field in changed:
                if field in possible_fields[key]:
                    possible_fields[key].remove(field)

Finall, we get the 1-to-1 correspondence:

In [15]:
possible_fields

defaultdict(<function __main__.<lambda>()>,
            {'departure location': {17},
             'departure station': {8},
             'departure platform': {10},
             'departure track': {11},
             'departure date': {0},
             'departure time': {19},
             'arrival location': {3},
             'arrival station': {2},
             'arrival platform': {9},
             'arrival track': {1},
             'class': {5},
             'duration': {4},
             'price': {14},
             'route': {6},
             'row': {15},
             'seat': {12},
             'train': {18},
             'type': {16},
             'wagon': {13},
             'zone': {7}})

To get the required answer (what do we get if we multiply the six fields started with the word "departure"):

In [16]:
field_of_interests = []
for field, idx in possible_fields.items():
    if field.startswith('departure'):
        field_of_interests.append(list(idx)[0])

ans = 1
nums_of_my_ticket = [int(num) for num in my_ticket.split(',')]
for i in field_of_interests:
    ans *= nums_of_my_ticket[i]

print(ans)    

4381476149273
